In [24]:
import torch
import numpy as np
from datasets import load_dataset, Features, Sequence, Value
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

MODEL_NAME = "distilbert-base-uncased"

language_categories = {
    "java": ["summary", "Ownership", "Expand", "usage", "Pointer", "deprecation", "rational"],
    "python": ["Usage", "Parameters", "DevelopmentNotes", "Expand", "Summary"],
    "pharo": ["Keyimplementationpoints", "Example", "Responsibilities", "Classreferences", "Intent", "Keymessages", "Collaborators"]
}

dataset_name = "NLBSE/nlbse25-code-comment-classification"
languages = ["java", "python", "pharo"]

all_results = {}

for lang in languages:
    all_labels = language_categories[lang]
    num_labels = len(all_labels)

    print(f"\n=== Processing language: {lang} ===")
    raw_ds = load_dataset(dataset_name)
    train_ds = raw_ds[f"{lang}_train"]
    test_ds = raw_ds[f"{lang}_test"]

    print("Columns in training set:", train_ds.column_names)
    print("First training example:", train_ds[0])
    print("All labels:", all_labels)

    label_map = {lbl: idx for idx, lbl in enumerate(all_labels)}
    inverse_label_map = {idx: lbl for lbl, idx in label_map.items()}

    print("label map:", label_map)

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    def encode_batch(examples):
        encodings = tokenizer(
            examples["comment_sentence"],
            truncation=True,
            padding=True,
            return_tensors="pt"
        )
        labels = torch.tensor(examples["labels"], dtype=torch.float32)
        return {
            "input_ids": encodings["input_ids"],
            "attention_mask": encodings["attention_mask"],
            "labels": labels
        }

    proc_train = train_ds.map(encode_batch, batched=True, batch_size=16)
    proc_test = test_ds.map(encode_batch, batched=True, batch_size=16)

    unused_columns = ['index', 'class', 'comment_sentence', 'partition', 'combo']
    proc_train = proc_train.remove_columns(unused_columns)
    proc_test = proc_test.remove_columns(unused_columns)

    features = Features({
        "input_ids": Sequence(Value("int64")),
        "attention_mask": Sequence(Value("int64")),
        "labels": Sequence(Value("float32"))
    })

    proc_train = proc_train.cast(features)
    proc_test = proc_test.cast(features)

    proc_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    proc_test.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

    print("num_labels:", num_labels)
    print("Shape of a sample's label vector:", proc_train[0]["labels"].shape)
    print("Dtype of a sample's label vector:", proc_train[0]["labels"].dtype)

    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=num_labels,
        problem_type="multi_label_classification"
    )

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        probs = 1 / (1 + np.exp(-predictions))
        preds = (probs > 0.5).astype(int)

        micro_precision, micro_recall, micro_f1, _ = precision_recall_fscore_support(
            labels, preds, average='micro', zero_division=0
        )
        acc = accuracy_score(labels, preds)

        per_cat_precision, per_cat_recall, per_cat_f1, _ = precision_recall_fscore_support(
            labels, preds, average=None, zero_division=0
        )

        per_category_metrics = {}
        for i, cat in enumerate(all_labels):
            per_category_metrics[f"{cat}_precision"] = per_cat_precision[i]
            per_category_metrics[f"{cat}_recall"] = per_cat_recall[i]
            per_category_metrics[f"{cat}_f1"] = per_cat_f1[i]

        metrics = {
            "accuracy": acc,
            "micro_precision": micro_precision,
            "micro_recall": micro_recall,
            "micro_f1": micro_f1
        }
        metrics.update(per_category_metrics)
        return metrics

    # (1 epoch for demonstration)
    training_args = TrainingArguments(
        output_dir=f"./{lang}_results",
        evaluation_strategy="no",
        save_strategy="no",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=1,
        learning_rate=5e-5,
        weight_decay=0.01,
        logging_dir=f"./{lang}_logs"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=proc_train,
        eval_dataset=proc_test,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    print("Starting training for language:", lang)
    trainer.train()
    print("Training done for language:", lang)

    print(f"Running evaluation on the test set for {lang}...")
    results = trainer.evaluate()
    print(f"Evaluation results for {lang}:", results)

    all_results[lang] = results

print("\n=== Summary of Results for All Languages ===")
for lang, res in all_results.items():
    print(f"Language: {lang}")
    for k, v in res.items():
        print(f"  {k}: {v}")
    print()


=== Processing language: java ===
Columns in training set: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels']
First training example: {'index': 0, 'class': 'Abfss.java', 'comment_sentence': 'azure blob file system implementation of abstractfilesystem.', 'partition': 0, 'combo': 'azure blob file system implementation of abstractfilesystem. | Abfss.java', 'labels': [1, 0, 0, 0, 0, 0, 0]}
All labels: ['summary', 'Ownership', 'Expand', 'usage', 'Pointer', 'deprecation', 'rational']
Label2ID: {'summary': 0, 'Ownership': 1, 'Expand': 2, 'usage': 3, 'Pointer': 4, 'deprecation': 5, 'rational': 6}


Map:   0%|          | 0/7614 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/7614 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1725 [00:00<?, ? examples/s]

...

Starting training for language: pharo


  0%|          | 0/163 [00:00<?, ?it/s]

{'train_runtime': 175.7398, 'train_samples_per_second': 7.386, 'train_steps_per_second': 0.928, 'train_loss': 0.36342822115845475, 'epoch': 1.0}
Training done for language: pharo
Running evaluation on the test set for pharo...


  0%|          | 0/37 [00:00<?, ?it/s]

Evaluation results for pharo: {'eval_loss': 0.27968451380729675, 'eval_accuracy': 0.36678200692041524, 'eval_micro_precision': 0.7371794871794872, 'eval_micro_recall': 0.38205980066445183, 'eval_micro_f1': 0.5032822757111597, 'eval_Keyimplementationpoints_precision': 0.0, 'eval_Keyimplementationpoints_recall': 0.0, 'eval_Keyimplementationpoints_f1': 0.0, 'eval_Example_precision': 0.732824427480916, 'eval_Example_recall': 0.8067226890756303, 'eval_Example_f1': 0.768, 'eval_Responsibilities_precision': 0.5384615384615384, 'eval_Responsibilities_recall': 0.1346153846153846, 'eval_Responsibilities_f1': 0.2153846153846154, 'eval_Classreferences_precision': 0.0, 'eval_Classreferences_recall': 0.0, 'eval_Classreferences_f1': 0.0, 'eval_Intent_precision': 1.0, 'eval_Intent_recall': 0.4, 'eval_Intent_f1': 0.5714285714285714, 'eval_Keymessages_precision': 0.0, 'eval_Keymessages_recall': 0.0, 'eval_Keymessages_f1': 0.0, 'eval_Collaborators_precision': 0.0, 'eval_Collaborators_recall': 0.0, 'eval_